# Simple Text Generation using RNN (LSTM) in TensorFlow

### 1. Import Libraries

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

### 2. Prepare the Dataset

In [11]:
# Sample text
text = "hello world this is a simple text generation example using rnn"

# Tokenize text into words
words = text.lower().split()
vocab = sorted(set(words))
word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for w, i in word2idx.items()}

# Create input-output pairs
sequences = []
for i in range(1, len(words)):
    seq = words[:i+1]
    encoded = [word2idx[w] for w in seq]
    sequences.append(encoded)

# Pad sequences and split input and label
max_len = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=len(vocab))

### 3. Build the RNN Model

In [12]:
model = Sequential()
model.add(Embedding(input_dim=len(vocab), output_dim=10, input_length=max_len-1))
model.add(LSTM(50))
model.add(Dense(len(vocab), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

### 4. Train the Model

In [13]:
model.fit(X, y, epochs=500, verbose=0)

### 5. Generate Text

In [14]:
def generate_text(seed_text, next_words=5):
    for _ in range(next_words):
        token_list = [word2idx.get(w, 0) for w in seed_text.split()]
        token_list = pad_sequences([token_list], maxlen=max_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        next_index = np.argmax(predicted)
        next_word = idx2word[next_index]
        seed_text += ' ' + next_word
    return seed_text

# Example usage
print(generate_text("you are good teacher"))

you are good teacher world world this is a
